# Fine-Tuning Open Source LLM for Customer Service Chatbot

In [1]:
# Imports
import os
import json
import torch
import evaluate
import torch.nn as nn
import transformers
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, pipeline
from peft import LoraConfig, get_peft_model
from datasets import Dataset, Features, ClassLabel, Value, Sequence
import warnings
warnings.filterwarnings('ignore')

c:\Users\krupc\Downloads\Projects\mlops\falcon-chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Checking the GPU
if torch.cuda.is_available():
    print('Number of GPUs:', torch.cuda.device_count())
    print('GPU Model: ', torch.cuda.get_device_name(0))
    print('Total Memory [GB] of GPU: ', round(torch.cuda.get_device_properties(0).total_memory / 1e9, 1))

Number of GPUs: 1
GPU Model:  NVIDIA GeForce RTX 4060 Ti
Total Memory [GB] of GPU:  8.6


## Defining Quantization Parameters
Simply put, these parameters are used to make the model fit into memory.

1. **`load_in_4bit=True`**
   - This parameter enables loading the model weights in **4-bit precision**. Quantization to 4-bit reduces the memory footprint of the model significantly, making it faster and more efficient for inference, especially in environments with limited GPU resources.

2. **`bnb_4bit_compute_dtype=torch.float16`**
   - Specifies the data type used for computations during 4-bit quantized operations. 
   - In this case, **`torch.float16`** (16-bit floating-point) is chosen. Using `float16` helps balance precision and performance, as it provides higher precision compared to integer types while still being efficient.

3. **`bnb_4bit_quant_type='nf4'`**
   - Defines the type of quantization to apply. **`nf4`** stands for "Normalized Floating-point 4-bit."
   - **`nf4`** is a specialized quantization format designed for neural network activations and weights. It offers better accuracy compared to traditional quantization techniques like `fp4` or `int4` because it normalizes the range of values, preserving more information.

4. **`bnb_4bit_use_double_quant=True`**
   - Enables **double quantization** for 4-bit weights.
   - Double quantization involves first quantizing the model weights into 4-bit values and then further compressing these values for storage and transfer. This can improve efficiency and reduce memory usage during training or inference.

5. **`llm_int8_enable_fp32_cpu_offload=True`**
   - Allows offloading specific computations to the CPU with **FP32 precision** (32-bit floating-point).
   - This is particularly useful for large language models (LLMs) when the GPU cannot handle certain tasks due to memory constraints. 
   - With this enabled, operations that require higher precision can offload to the CPU, ensuring no loss in accuracy while optimizing GPU memory usage.


In [4]:
# Defining the paramaters
quantization_config = BitsAndBytesConfig(load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.float16,
                                         bnb_4bit_quant_type='nf4',
                                         bnb_4bit_use_double_quant=True,
                                         llm_int8_enable_fp32_cpu_offload=True)

---

## Load the Model and the Tokenizer

https://huggingface.co/tiiuae/falcon-7b

In [8]:
# Model
model = AutoModelForCausalLM.from_pretrained('tiiuae/falcon-7b',
                                             quantization_config=quantization_config,
                                             device_map='auto')

print(model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.38s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


FalconForCausalLM(
  (transformer): FalconModel(
    (word_embeddings): Embedding(65024, 4544)
    (h): ModuleList(
      (0-31): 32 x FalconDecoderLayer(
        (self_attention): FalconAttention(
          (query_key_value): Linear4bit(in_features=4544, out_features=4672, bias=False)
          (dense): Linear4bit(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
          (rotary_emb): FalconRotaryEmbedding()
        )
        (mlp): FalconMLP(
          (dense_h_to_4h): Linear4bit(in_features=4544, out_features=18176, bias=False)
          (act): GELUActivation()
          (dense_4h_to_h): Linear4bit(in_features=18176, out_features=4544, bias=False)
        )
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
    (rotary_emb): FalconRotaryEmbedding()
  )
  (lm_head): Linear(in_features=4544, out_features=65024, bi

In [9]:
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained('tiiuae/falcon-7b')

print(tokenizer)

PreTrainedTokenizerFast(name_or_path='tiiuae/falcon-7b', vocab_size=65024, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|endoftext|>', 'additional_special_tokens': ['>>TITLE<<', '>>ABSTRACT<<', '>>INTRODUCTION<<', '>>SUMMARY<<', '>>COMMENT<<', '>>ANSWER<<', '>>QUESTION<<', '>>DOMAIN<<', '>>PREFIX<<', '>>SUFFIX<<', '>>MIDDLE<<']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken(">>TITLE<<", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken(">>ABSTRACT<<", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken(">>INTRODUCTION<<", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken(">>SUMMARY<<", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken(">>COMMENT<<", rstrip=False, lstrip=False, single_word=False, normalized=False, 

## Freezing the Original Weights

**`param.requires_grad = False`**: This disables gradient computation for each parameter, which means these parameters will not be updated during training. This is commonly done when freezing certain parts of a pre-trained model for transfer learning, where only the top layers of the model are trained.

**`if param.ndim == 1: param.data = param.data.to(torch.float32)`**: This line converts one-dimensional parameters (i.e., vectors) to the `float32` data type. This might be necessary to ensure that all model parameters are of the same data type, especially when performing operations that require consistent data types.

In [10]:
for param in model.parameters():
    param.requires_grad = False
    if param.ndim == 1:
        param.data = param.data.to(torch.float16)

## Enabling the Checkpoint of the Model

In [11]:
# Enables the gradient checkpoint feature in the model
model.gradient_checkpointing_enable()

When gradient checkpointing is enabled, the model does not store all intermediate values (layer activations) during the forward pass. Instead, it saves only a few checkpointed values. During the backward pass, the intermediate values that were not stored are recomputed from the checkpoints. This reduces the amount of memory required to store intermediate values but increases computation time because some values need to be recomputed.

This technique is useful when training very large models that would otherwise not fit into GPU memory.

In [12]:
# Enables the technique called "gradient checkpointing"
model.enable_input_require_grads()

This technique is useful for reducing memory consumption during the training of large models. It works by saving certain intermediate states (checkpoints) during the forward pass and then using these states during the backward pass to recompute the gradients instead of storing all intermediate states in memory. This can reduce the amount of memory required but may increase computation time.

## Adjusting Conversion to Tensor

The CastOutputToFloat class is a subclass of PyTorch's nn.Sequential class, which is used to create a sequence of modules (like layers of a neural network). The main functionality of this class is to convert the data type of the output of a sequence of modules to torch.float32 (i.e., a 32-bit floating-point tensor).

In [13]:
# Tensor Conversion
class CastOutputFloat(nn.Sequential):
    def forward(self, input):
        return super().forward(input).to(torch.float32)

The code below is replacing the language model head (lm_head) with the CastOutputToFloat module that wraps the original language model head. This means that every time the model's lm_head is used during the forward pass, its output will automatically be converted to torch.float32. This can be useful for ensuring type compatibility in situations where the output of the language model head needs to be of type float32.

In [14]:
model.lm_head = CastOutputFloat(model.lm_head)

## Setting Fine Tuning Parameters

**LoRA**

LoRA (Low-Rank Adaptation) is a technique designed to make fine-tuning large-scale machine learning models more efficient and computationally feasible. It achieves this by introducing low-rank matrices into the model, allowing for parameter-efficient fine-tuning while maintaining performance comparable to traditional methods.

Let me break down the parameters in the `LoraConfig` configuration:

1. **`r = 16`**
   - This specifies the **rank** of the low-rank adaptation (LoRA) matrices.
   - LoRA replaces parts of the weight matrices in a neural network with low-rank matrices to reduce the number of trainable parameters.
   - A higher rank (`r`) increases the model's capacity but also increases the number of trainable parameters.

2. **`lora_alpha = 32`**
   - This is a scaling factor used in the LoRA mechanism.
   - The output of the LoRA weights is scaled by `lora_alpha / r`.
   - It controls the contribution of the LoRA layers to the final output, allowing fine-tuning of the impact of the low-rank updates.

3. **`lora_dropout = 0.05`**
   - Specifies the dropout rate for the LoRA layers.
   - Dropout randomly zeroes out some connections in the LoRA layers during training to prevent overfitting.
   - A value of `0.05` means there is a 5% chance of a connection being dropped during training.

4. **`bias = "none"`**
   - Indicates whether biases in the model should be adapted.
   - Options include:
     - `"none"`: No biases are adapted.
     - `"all"`: All biases in the model are adapted.
     - `"lora_only"`: Only biases in LoRA layers are adapted.

5. **`task_type = "CAUSAL_LM"`**
   - Specifies the type of task the model is being fine-tuned for.
   - In this case, `"CAUSAL_LM"` stands for **Causal Language Modeling**, which involves predicting the next token in a sequence (as in autoregressive models like GPT).
   - Other possible task types might include `"SEQ2SEQ_LM"` (sequence-to-sequence language modeling) or `"TOKEN_CLASSIFICATION"`, depending on the specific LoRA implementation.

Summary
This `LoraConfig` is designed to apply LoRA to a model in a way that reduces the number of trainable parameters, making fine-tuning more efficient. It's specifically configured for causal language modeling tasks, with a moderate rank (`r=16`), a scaling factor (`lora_alpha=32`), and a small amount of dropout (`lora_dropout=0.05`) to balance between regularization and performance. The bias is not adapted in this setup.

In [15]:
# LoRa Config
config = LoraConfig(r = 16,
                    lora_alpha = 32,
                    lora_dropout = 0.05,
                    bias = "none",
                    task_type = "CAUSAL_LM")

In [16]:
# Creates the model considering the LoRA parameters
model = get_peft_model(model, config)

In [17]:
# Defining the function to print the trainable parameters of a model
def print_trainable_parameters(model):

    # Initialize the count of trainable parameters
    trainable_params = 0

    # Initialize the count of all parameters
    all_param = 0

    # Iterate over all named parameters of the model
    for _, param in model.named_parameters():

        # Sum the total number of elements of all parameters
        all_param += param.numel()

        # Check if the parameter is trainable
        if param.requires_grad:

            # Sum the number of elements of the trainable parameters
            trainable_params += param.numel()

    # Print the number of trainable parameters, the total number of parameters, and the percentage of trainable parameters
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

In [18]:
# Execute the function
print_trainable_parameters(model)

trainable params: 4718592 || all params: 5477012352 || trainable%: 0.08615266310796153
